# Chapter 6: Tasks
`ldmunit` provides implementations of model recovery and parameter recovery tasks. Here we shortly explain these two tasks and show simple example code and results.

In [1]:
import sciunit
from os import getcwd
from ldmunit.models.decision_making import NWSLSModel, RWCKModel, RWModel
from ldmunit.tasks import model_recovery, param_recovery
from ldmunit.testing import InteractiveTest
from ldmunit.scores import NLLScore
from ldmunit.utils import partialclass
from ldmunit.envs import BanditEnv
sciunit.settings['CWD'] = getcwd()

## Model Recovery
Model recovery is the task of generating simulated data from a sequence of models, from one at a time, and then testing all the models in this sequence against this data to see which models fits best. If we have `N` models, this task generates `N * N` many score values, `N` score values for each of the simulated data from the corresponding model. This allows us to represents the results succinctly as a score matrix. `ldmunit` provides `ldmunit.tasks.model_recovery` function for this purpose, which we demonstrate below:

In [2]:
p_dist = [0.2, 0.5, 0.3, 0.1]
n_action = len(p_dist)
env = BanditEnv(p_dist=p_dist)
test_cls = partialclass(InteractiveTest, score_type=partialclass(NLLScore, min_score=0, max_score=1e3))
test_suite, score_matrix = model_recovery(
    [NWSLSModel(n_action=n_action, n_obs=1), RWCKModel(n_action=n_action, n_obs=1), RWModel(n_action=n_action, n_obs=1)],
    env,
    test_cls,
    n_trials=75,
    seed=42
)
display(score_matrix)

01-29-20 11:44 ldmunit      INFO     model_recovery : Simulating model NWSLSModel against env BanditEnv
01-29-20 11:44 ldmunit      INFO     model_recovery : Simulating model RWCKModel against env BanditEnv
01-29-20 11:44 ldmunit      INFO     model_recovery : Simulating model RWModel against env BanditEnv


01-29-20 11:44 ldmunit      INFO     Ground truth: NWSLSModel : Optimizing NWSLSModel model...


01-29-20 11:44 ldmunit      INFO     Ground truth: RWCKModel : Optimizing NWSLSModel model...


01-29-20 11:44 ldmunit      INFO     Ground truth: RWModel : Optimizing NWSLSModel model...


01-29-20 11:44 ldmunit      INFO     Ground truth: NWSLSModel : Optimizing RWCKModel model...


...

01-29-20 11:44 ldmunit      INFO     Ground truth: RWModel : Optimizing RWCKModel model...


01-29-20 11:44 ldmunit      INFO     Ground truth: NWSLSModel : Optimizing RWModel model...


01-29-20 11:44 ldmunit      INFO     Ground truth: RWCKModel : Optimizing RWModel model...


01-29-20 11:44 ldmunit      INFO     Ground truth: RWModel : Optimizing RWModel model...


,Ground truth: NWSLSModel,Ground truth: RWCKModel,Ground truth: RWModel
NWSLSModel,69.7,120,118
RWCKModel,104,103,104
RWModel,104,103,104


## Parameter Recovery
Parameter recovery is the task of simulating data from a given model with a sequence of parameter settings, one at a time, and then fitting the model on the generated data to see if the fitted parameter models are similar to the ones used to generate the data in the first place. Since the fits are subject to randomness, data simulation and model fitting is performed multiple times for a single parameter setting. Therefore, if we have `K` parameter settings and we perform `T` fits for each setting, we will have `K * T` many fitted parameter values as the result of this task. `ldmunit` offers parameter recovery implementation with `ldmunit.tasks.parameter_recovery` function, which we demonstrate below:

In [4]:
param_settings = [
    {'epsilon': 0},
    {'epsilon': 2},
    {'epsilon': 3}
]
results = param_recovery(
    param_settings,
    NWSLSModel(n_action=n_action, n_obs=1), 
    env,
    n_runs=3,
    n_trials=75,
)
for setting, its_results in zip(param_settings, results):
    print('Original setting:', setting)
    print('Fitted results')
    print(its_results)
    print()

01-29-20 11:45 ldmunit      INFO     param_recovery: Recovering parameters with index 0
01-29-20 11:45 ldmunit      INFO     param_recovery: Recovering parameters with index 1
01-29-20 11:45 ldmunit      INFO     param_recovery: Recovering parameters with index 2


Original setting: {'epsilon': 0}
Fitted results
[{'epsilon': 0.8999999999999999}, {'epsilon': 1.7999999999999998}, {'epsilon': 2.6999999999999993}]

Original setting: {'epsilon': 2}
Fitted results
[{'epsilon': 1.1500000000000004}, {'epsilon': 2.05}, {'epsilon': 2.6999999999999993}]

Original setting: {'epsilon': 3}
Fitted results
[{'epsilon': 0.00075}, {'epsilon': 3.075}, {'epsilon': 2.3000000000000007}]

